In [1]:
cd ../src/

/Users/brent/Archiver/Workspace/qcqp/src


In [2]:
import collections


import numpy as np
import pandas as pd

###########
# display options
###########


pd.set_option("display.max_columns", None)
np.set_printoptions(
  linewidth=200,
  precision=4
)

from pyqp import bg_grb, bg_msk, bg_msk_msc, bg_msk_chordal, bg_msk_homo
from pyqp import bb_msc, bb, bb_diag, bb_socp
from pyqp.classes import QP, QPI, Bounds, BCParams
from pyqptest.helpers import *

In [3]:
parser.print_help()
args, unknown = parser.parse_known_args()
params = BCParams()
kwargs, r_methods = params.produce_args(parser, METHOD_CODES)

usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] [--r R]
                   [--fpath FPATH] [--n N] [--m M] [--pc PC]
                   [--time_limit TIME_LIMIT] [--verbose VERBOSE] [--bg BG]
                   [--bg_rd BG_RD]

optional arguments:
  -h, --help            show this help message and exit
  --dump_instance DUMP_INSTANCE
                        if save instance
  --r R                 {1: 'grb', 2: 'shor', 3: 'dshor', 4: 'msc', 5: 'emsc',
                        6: 'ssdp', 7: 'bb', 8: 'bb_msc', 9: 'shor_homo'}
  --fpath FPATH         path of the instance
  --n N                 dim of x
  --m M                 if randomly generated num of constraints
  --pc PC               if randomly generated problem type
  --time_limit TIME_LIMIT
                        time limit of running.
  --verbose VERBOSE     if verbose
  --bg BG               backend used
  --bg_rd BG_RD         backend used, rank reduction
usage: QCQP runner [-h] [--dump_instance DUMP_INSTANCE] [--r R]

In [4]:
r_methods = {'bb', 'grb', 'shor'}

In [5]:
n, m = 8, 2
# problem
problem_id = f"{n}:{m}:{0}"
# start
# qp = QPI.block(n, m, r=2, eps=0.5)
qp = QPI.normal(int(n), int(m), rho=0.2)
bd = Bounds(xlb=np.zeros(shape=(n, 1)), xub=np.ones(shape=(n, 1)))

# benchmarking

In [7]:
evals = []
results = {}
# run methods
for k in r_methods:
  func = METHODS[k]
  qp1 = bb_msc.QP(*qp.unpack())
  qp1.decompose()
  r = func(qp1, bd, params=params)
  reval = r.eval(problem_id)
  evals.append({**reval.__dict__, "method": k})
  results[k] = r
for k, r in results.items():
  print(f"{k} benchmark @{r.relax_obj}")
  r.check(qp)

Using license file /Users/brent/licenses/gurobi.lic
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 1 threads
Optimize a model with 0 rows, 8 columns and 0 nonzeros
Model fingerprint: 0x8f6466d0
Model has 9 quadratic objective terms
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 2e+01]
  QLMatrix range   [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [4e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+01, 2e+01]

Continuous model is non-convex --

In [8]:
results['grb'].xval, results['bb'].xval.round(2)

(array([[0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.]]),
 array([[ 0.18],
        [ 1.  ],
        [-0.  ],
        [ 1.  ],
        [ 1.  ],
        [-0.  ],
        [ 1.  ],
        [ 1.  ]]))

In [9]:
xval = results['grb'].xval
sval = (xval.T@xval).trace()
xval, sval

(array([[0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.]]),
 5.0)

In [10]:
import pyqp.bg_msk_admm as pyadm
import importlib
importlib.reload(pyadm)

<module 'pyqp.bg_msk_admm' from '/Users/brent/Archiver/Workspace/qcqp/src/pyqp/bg_msk_admm.py'>

In [11]:
qp1 = bb_msc.QP(*qp.unpack())
qp1.decompose(decompose_method='eig-type2')

In [12]:
# initialize
m, n, dim = qp1.a.shape
xval = np.ones((n, dim))
sval = (xval.T @ xval).trace()
rho = 1
kappa = 0
mu = 0
xival = xval

In [13]:
# test run
max_iter = 1e1
_iter = 0
while True:
  _iter += 1
  r = pyadm.msc_subproblem_x(xival,
                             sval,
                             rho,
                             kappa,
                             rho,
                             qp1,
                             bd,
                             solve=False,
                             verbose=False)
  r.solve()
  tval = r.t.level()[0]
  r_xi = pyadm.msc_subproblem_xi(r.xval,
                                 r.yval,
                                 r.zval,
                                 tval,
                                 kappa,
                                 mu,
                                 rho,
                                 qp1,
                                 bd,
                                 solve=False,
                                 verbose=False)
  r_xi.problem.solve()
  sval = r_xi.s.level()[0]
  xival = r_xi.xivar.level().reshape(r_xi.xivar.getShape())
  residual_ts = tval - sval
  residual_xix = (xival * r.xval).sum() - sval
  print(
    f"iteration @{_iter}: norm t - s: {residual_ts}, 𝜉x - s: {residual_xix}")
  if max(residual_ts, residual_xix) < 1e-4:
    break
  if _iter > max_iter:
    break


Problem
  Name                   : many_small_cone_msk
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 62              
  Cones                  : 2               
  Scalar variables       : 18              
  Matrix variables       : 24              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : many_small_cone_msk
  Objective sense        : min             
  Type                   : CONIC (conic opt

In [14]:
xival, r.xval

(array([[0.9988],
        [1.    ],
        [0.    ],
        [1.    ],
        [1.    ],
        [0.    ],
        [1.    ],
        [1.    ]]),
 array([[ 0.286 ],
        [ 0.5196],
        [-0.    ],
        [ 0.912 ],
        [ 0.567 ],
        [-0.    ],
        [ 0.5001],
        [ 0.7563]]))